# *Local Food Economics Data Visualization Virtual Challenge*

**Authors:** ***Manuel Garcia***, ***Oscar Sarasty***, ***and*** **_Trevor Johnson_** 

**Group V4**

**Recomended citation:** [Garcia, M., Sarasty, O., Johnson, T.]. [(2023)]. [FI Index Local Food Economics Analysis, (Version 4)]. USDA AMS Data Warehouse. [https://github.com/manueljgarcia/USDA-AMS-Data-and-Metrics.git].

#### Libraries needed from the our project

In [2]:
import pandas as pd
#!pip install statsmodels
import statsmodels.formula.api as smf
import statsmodels.api as sm
#!pip install scikit-learn
from sklearn.linear_model import LinearRegression

#### Datasets

In [3]:
analytic_data = pd.read_csv('C:/Users/gar14685/OneDrive - Texas Tech University/PHD PROGRAM/CONFERENCES/AAEA 2023/Data Viz Competition/data/dataset_viz/analytic_data2019.csv')

'''analytic_data is a public dataset available in the County Health Rankings from 2019, it can be found at: 
https://www.countyhealthrankings.org/explore-health-rankings/rankings-data-documentation/national-data-documentation-2010-2019'''

food_access1 = pd.read_csv('C:/Users/gar14685/OneDrive - Texas Tech University/PHD PROGRAM/CONFERENCES/AAEA 2023/Data Viz Competition/data/food_access/food_access1.csv')
food_access2 = pd.read_csv('C:/Users/gar14685/OneDrive - Texas Tech University/PHD PROGRAM/CONFERENCES/AAEA 2023/Data Viz Competition/data/food_access/food_access2.csv')
grants = pd.read_csv('C:/Users/gar14685/OneDrive - Texas Tech University/PHD PROGRAM/CONFERENCES/AAEA 2023/Data Viz Competition/data/grants/grants.csv')

#### Filter of datasets based on the year 2019 and the required variables

In [4]:
# Filter datasets based on the year and the required variables
food_access1_filtered = food_access1[(food_access1['year'] == 2019) & 
                                     (food_access1['variable_name'].isin(['child_food_insecurity_rate', 
                                                                           'food_insecurity_rate', 
                                                                           'mean_usual_hours', 
                                                                           'SNAP_percent_white_alone_not_hispanic']))]

food_access2_filtered = food_access2[(food_access2['year'] == 2019) & 
                                     (food_access2['variable_name'].isin(['child_food_insecurity_rate', 
                                                                           'food_insecurity_rate', 
                                                                           'mean_usual_hours', 
                                                                           'SNAP_percent_white_alone_not_hispanic']))]

grants_filtered = grants[(grants['year'] == 2019) & 
                         (grants['variable_name'] == 'food_desert_vehicle')]


#### Pivot the filtered datasets

In [5]:
# Pivot the filtered datasets to prepare them for merging
food_access1_pivoted = food_access1_filtered.pivot(index='fips', columns='variable_name', values='value').reset_index()
food_access2_pivoted = food_access2_filtered.pivot(index='fips', columns='variable_name', values='value').reset_index()
grants_pivoted = grants_filtered.pivot(index='fips', columns='variable_name', values='value').reset_index()

#### Merging the datasets

In [6]:
# Merge the datasets
merged_data = pd.merge(food_access1_pivoted, food_access2_pivoted, on='fips', how='outer')
merged_data = pd.merge(merged_data, grants_pivoted, on='fips', how='outer')

analytic_data2019_filtered = analytic_data[analytic_data['year'] == 2019]

# Merge analytic_data2019 with the final dataset
merged_data_all = pd.merge(merged_data, analytic_data2019_filtered, left_on='fips', right_on='fipscode', how='outer')

# Remove rows where 'fips' is less than 100 and where 'state' is empty or NaN
merged_data_all_filtered = merged_data_all[(merged_data_all['fips'] >= 100) & (merged_data_all['state'].notna())]

#### Saving the merged final dataset in .csv format

In [7]:
# Save the updated data to a CSV file
merged_data_all_filtered.to_csv('C:/Users/gar14685/OneDrive - Texas Tech University/PHD PROGRAM/CONFERENCES/AAEA 2023/Data Viz Competition/data_output/merged_data.csv', index=False)

merged_data_all_filtered.head()

,fips,child_food_insecurity_rate,food_insecurity_rate,mean_usual_hours,SNAP_percent_white_alone_not_hispanic,food_desert_vehicle,statecode,countycode,fipscode,state,...,v013_rawvalue,v016_rawvalue,v017_rawvalue,v025_rawvalue,v026_rawvalue,v027_rawvalue,v031_rawvalue,v032_rawvalue,v035_rawvalue,v097_rawvalue
51,1001.0,0.196,0.157,40.5,0.509,1.0,1.0,1.0,1001.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,1003.0,0.144,0.134,39.4,0.730,3.0,1.0,3.0,1003.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,1005.0,0.324,0.207,38.8,0.242,3.0,1.0,5.0,1005.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,1007.0,0.218,0.163,40.0,0.637,1.0,1.0,7.0,1007.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,1009.0,0.154,0.145,41.0,0.832,2.0,1.0,9.0,1009.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Looking for missing values in our dataset

In [9]:
# List of variables in your formula
vars_in_formula = ['food_insecurity_rate', 'v083_rawvalue', 'v003_rawvalue',
                   'v065_rawvalue', 'mean_usual_hours', 'food_desert_vehicle', 
                   'SNAP_percent_white_alone_not_hispanic']

# Print the number of unique and NaN values for each variable
for var in vars_in_formula:
    print(f"{var}: unique values - {merged_data_all_filtered[var].nunique()}, NaN values - {merged_data_all_filtered[var].isna().sum()}")

food_insecurity_rate: unique values - 210, NaN values - 0
v083_rawvalue: unique values - 3114, NaN values - 19
v003_rawvalue: unique values - 3135, NaN values - 1
v065_rawvalue: unique values - 2962, NaN values - 121
mean_usual_hours: unique values - 143, NaN values - 1
food_desert_vehicle: unique values - 67, NaN values - 0
SNAP_percent_white_alone_not_hispanic: unique values - 871, NaN values - 1


#### Predicting the missing values in our dataset

In [10]:
# Define the predictor variables for the model
predictors = ['v137_rawvalue', 'v153_rawvalue', 'v051_rawvalue', 'v052_rawvalue',
              'v053_rawvalue', 'v054_rawvalue', 'v056_rawvalue', 'v059_rawvalue']

# Variables to be predicted
target_vars = ['v003_rawvalue', 'v083_rawvalue', 'v065_rawvalue']

for target in target_vars:
    # Split the data into training set and to-be-predicted set
    train_data = merged_data_all_filtered[merged_data_all_filtered[target].notna()]
    predict_data = merged_data_all_filtered[merged_data_all_filtered[target].isna()]
    
    # Check if there are any rows in the prediction dataset
    if predict_data.shape[0] > 0:
        # Fit the model using the training data
        model = LinearRegression()
        model.fit(train_data[predictors], train_data[target])
        
        # Predict the missing values
        predictions = model.predict(predict_data[predictors])
        
        # Replace missing values with the predictions
        merged_data_all_filtered.loc[merged_data_all_filtered[target].isna(), target] = predictions
    else:
        print(f"No rows to predict for target variable {target}")

merged_data_all_filtered.head()

'''
Predictors:
Population raw value	                    v051_rawvalue
% below 18 years of age raw value	        v052_rawvalue
% 65 and older raw value	                v053_rawvalue
% Non-Hispanic African American raw value	v054_rawvalue
% Hispanic raw value	                    v056_rawvalue
% not proficient in English raw value	    v059_rawvalue
Long commute - driving alone raw value	    v137_rawvalue
Homeownership raw value	                    v153_rawvalue

Variables to be predicted:
Uninsured adults raw value	                                v003_rawvalue
Children eligible for free or reduced price lunch raw value	v065_rawvalue
Limited access to healthy foods raw value	                v083_rawvalue

'''

,fips,child_food_insecurity_rate,food_insecurity_rate,mean_usual_hours,SNAP_percent_white_alone_not_hispanic,food_desert_vehicle,statecode,countycode,fipscode,state,...,v013_rawvalue,v016_rawvalue,v017_rawvalue,v025_rawvalue,v026_rawvalue,v027_rawvalue,v031_rawvalue,v032_rawvalue,v035_rawvalue,v097_rawvalue
51,1001.0,0.196,0.157,40.5,0.509,1.0,1.0,1.0,1001.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,1003.0,0.144,0.134,39.4,0.730,3.0,1.0,3.0,1003.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53,1005.0,0.324,0.207,38.8,0.242,3.0,1.0,5.0,1005.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54,1007.0,0.218,0.163,40.0,0.637,1.0,1.0,7.0,1007.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,1009.0,0.154,0.145,41.0,0.832,2.0,1.0,9.0,1009.0,AL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Estimating our new food insecurity index

In [11]:
# Define the formula for the GLM
formula = 'food_insecurity_rate ~ v083_rawvalue + v003_rawvalue + v065_rawvalue + mean_usual_hours + food_desert_vehicle + SNAP_percent_white_alone_not_hispanic'

# Fit the GLM model
glm_model = sm.GLM.from_formula(formula, data=merged_data_all_filtered, family=sm.families.Gaussian(link=sm.families.links.identity()))
glm_result = glm_model.fit()

c:\Users\gar14685\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\genmod\families\links.py:13: FutureWarning: The identity link alias is deprecated. Use Identity instead. The identity link alias will be removed after the 0.15.0 release.
  warnings.warn(


In [12]:
# Extract the predicted values from the GLM model
predicted_values = pd.Series(glm_result.predict(), name='predicted_food_insecurity_rate')
merged_data_all_filtered = merged_data_all_filtered.reset_index(drop=True)

# Assign the predicted values to the new column
merged_data_all_filtered['predicted_food_insecurity_rate'] = predicted_values

#### Saving our final dataset with the new food insecurity index estimated

In [13]:
# Save the updated data with predicted values to a new CSV file
merged_data_all_filtered.to_csv('C:/Users/gar14685/OneDrive - Texas Tech University/PHD PROGRAM/CONFERENCES/AAEA 2023/Data Viz Competition/data_output/new_fi_index.csv', index=False)

### Data viz created uising Tableau

##### The data viz project was created using a Map to visualize the new Food insecurity index by county and region

#### The constructed index is presented in our data visualization through a red-green color gradient shown on a county basis. Food insecurity, as defined by our index, increases with the darkness of the red color. That is, the darker the red color of the county is on the visualization, the more food insecure the county is. 
